In [1]:
import os

In [2]:
pwd

'd:\\CloudDeplyment\\Image-Segmentation-with-AWS\\reserach'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\CloudDeplyment\\Image-Segmentation-with-AWS'

In [5]:
## Create Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from VGGClassifier.constants import CONFIG_FILE_PATH

In [7]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [8]:
# Read yaml file
# Configuration File

from VGGClassifier.constants import *
from VGGClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from VGGClassifier.utils.common import *


In [24]:

class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,param_filepath= PARAMS_FILE_PATH):
        # reading configuration file and parameter file
        self.config = read_yaml(config_file_path)
    
        self.param = read_yaml(param_filepath)
        # creating directories with the name in artifacts_root i.e. artifacts
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


  

In [19]:
# Components for data ingestion

import os
import urllib.request as request
import zipfile
from VGGClassifier import logger
from VGGClassifier.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [27]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    print("configuration loaded successfully", data_ingestion_config)


    data_ingestion = DataIngestion(config=data_ingestion_config)
    print("Data ingestion files", data_ingestion)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-03 15:10:09,969- INFO]- common - yaml file: config\config.yaml loaded successfully
[2023-08-03 15:10:09,973- INFO]- common - yaml file: params.yaml loaded successfully
[2023-08-03 15:10:09,978- INFO]- common - created directory at: artifacts
[2023-08-03 15:10:09,980- INFO]- common - created directory at: artifacts/data_ingestion


configuration loaded successfully DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
Data ingestion files <__main__.DataIngestion object at 0x0000029527831250>
[2023-08-03 15:10:13,660- INFO]- 1170291011 - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EF52:1732:336811:34C4A8:64CBA4BD
Accept-Ranges: bytes
Date: Thu, 03 Aug 2023 13:10:11 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-